In [61]:
# See https://github.com/deep-learning-with-pytorch/dlwpt-code/blob/master/p1ch8/1_convolution.ipynb

%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [40]:
class_names = ['anger','disgust','fear','joy','neutral',
               'sadness','surprise']

In [41]:
df = pd.read_csv('../data/FERG_DB_256.csv')

In [42]:
df['emotion'] = df['emotion'].astype('category')

In [43]:
df.head()

,name,emotion,number
0,aia,anger,1
1,aia,anger,10
2,aia,anger,100
3,aia,anger,1000
4,aia,anger,1001


See https://github.com/deep-learning-with-pytorch/dlwpt-code/blob/master/p1ch8/1_convolution.ipynb

In [63]:
def get_filename(row):
   name = row['name']
   emotion = row['emotion']
   number = str(row['number'])
   
   return f'/users/9/reine097/projects/auto-label-emotions/data/FERG_DB_256/{name}/{name}_{emotion}/{name}_{emotion}_{number}.png'

In [64]:
df['file_path'] = df.apply(get_filename, axis=1)
df.head()

,name,emotion,number,file_path
0,aia,anger,1,/users/9/reine097/projects/auto-label-emotions/data/FERG_DB_256/aia/aia_anger/aia_anger_1.png
1,aia,anger,10,/users/9/reine097/projects/auto-label-emotions/data/FERG_DB_256/aia/aia_anger/aia_anger_10.png
2,aia,anger,100,/users/9/reine097/projects/auto-label-emotions/data/FERG_DB_256/aia/aia_anger/aia_anger_100.png
3,aia,anger,1000,/users/9/reine097/projects/auto-label-emotions/data/FERG_DB_256/aia/aia_anger/aia_anger_1000.png
4,aia,anger,1001,/users/9/reine097/projects/auto-label-emotions/data/FERG_DB_256/aia/aia_anger/aia_anger_1001.png


In [68]:
X = list(df['file_path'])
y = np.array(list(df['emotion']))

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [52]:
connected_model = nn.Sequential(
            nn.Linear(3072, 1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 128),
            nn.Tanh(),
            nn.Linear(128, 2))

In [53]:
numel_list = [p.numel()
              for p in connected_model.parameters()
              if p.requires_grad == True]
sum(numel_list), numel_list

(3737474, [3145728, 1024, 524288, 512, 65536, 128, 256, 2])

In [54]:
first_model = nn.Sequential(
                nn.Linear(3072, 512),
                nn.Tanh(),
                nn.Linear(512, 2),
                nn.LogSoftmax(dim=1))

In [55]:
numel_list = [p.numel() for p in first_model.parameters()]
sum(numel_list), numel_list

(1574402, [1572864, 512, 1024, 2])

In [56]:
linear = nn.Linear(256, 256)

linear.weight.shape, linear.bias.shape

(torch.Size([256, 256]), torch.Size([256]))

In [57]:
conv = nn.Conv2d(3, 16, kernel_size=3) # <1>
conv

Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))

In [58]:
conv.weight.shape, conv.bias.shape

(torch.Size([16, 3, 3, 3]), torch.Size([16]))

In [71]:
image_path = X_train[0]
try:
    img = Image.open(image_path)
except FileNotFoundError:
    print(f"Error: Image file not found at {image_path}")
except Exception as e:
        print(f"An error occurred: {e}")
else:
    # Further operations with the image object (e.g., display, save, process)
    img.show() #display the image